In [1]:
import pandas as pd
from glob import glob
import torch
import torch.nn as nn
import segmentation_models_pytorch as smp
import numpy as np

/home/ebneto/miniconda3/envs/pl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
KWH_COST = 0.85

In [3]:
def calculate_flops(model, input_size=(3, 256, 256)): # number of channels is irrelevant in this function
    flops = 0
    input = torch.randn(1, *input_size).to(next(model.parameters()).device)

    for layer in model.modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.ConvTranspose2d):
            out_h = (input_size[1] - layer.kernel_size[0]) // layer.stride[0] + 1
            out_w = (input_size[2] - layer.kernel_size[1]) // layer.stride[1] + 1
            flops += 2 * layer.in_channels * layer.out_channels * layer.kernel_size[0] * layer.kernel_size[1] * out_h * out_w / layer.stride[0]
            input_size = (layer.out_channels, out_h, out_w)
        elif isinstance(layer, nn.Linear):
            flops += 2 * layer.in_features * layer.out_features
        elif isinstance(layer, nn.ReLU):
            continue  # ReLU doesn't involve FLOPs
        else:
            # print(f"Warning: layer {layer} not counted")
            pass

    return flops

In [4]:
df = []
gpumetrics_paths = sorted(glob('results/*.csv'))
results_paths = sorted(glob('results/*.txt'))

for result_file, gpu_file in zip(results_paths, gpumetrics_paths):
    # Get GPU data
    gpu_data = pd.read_csv(gpu_file)
    encoder = gpu_file.split('-')[1]
    avg_power = gpu_data['Power (W)'].mean()
    time = gpu_data['Time (s)'].max()
    avg_mem = gpu_data['Memory Usage (MiB)'].mean()

    energy_kwh = avg_power * time / 1000 / 3600
    
    # Loading the results
    with open(result_file, 'r') as f:
        result = f.read()
    lines = result.split('\n')

    # Extract the relevant information
    composition = lines[1].split('-')[1]
    precision = float(lines[2].split(': ')[1])
    f1_score = float(lines[3].split(': ')[1])
    iou = float(lines[4].split(': ')[1])
    accuracy = float(lines[5].split(': ')[1])
    recall = float(lines[6].split(': ')[1])
    epoch = int(lines[7].split(': ')[1])
    time = lines[8].split(': ')[1]

    # Treating some results
    encoder = encoder.replace('efficientnet', 'efficientnet-b0')
    in_channels = len(str(composition)) if composition.isnumeric() else 8

    # Instantiating model to calculate FLOPs
    model = smp.DeepLabV3Plus(encoder_name = encoder, in_channels=in_channels, classes=1)
    gflops = np.round(calculate_flops(model) / 1e9, 4) # Considering standard input size of 256x256

    # Save results to a dict
    results = {
        'encoder': encoder,
        'composition': composition,
        'precision': precision,
        'f1_score': f1_score,
        'iou': iou,
        'accuracy': accuracy,
        'recall': recall,
        'epoch': epoch,
        'time': time,
        'avg_power': avg_power,
        'avg_mem': avg_mem,
        'GFLOPs': gflops,
        'energy_kwh': energy_kwh,
        'estimated cost (R$)': energy_kwh * KWH_COST * 2
    }

    df.append(results)

df = pd.DataFrame(df)
df = df.sort_values(by=['iou'], ascending=False)
df

,encoder,composition,precision,f1_score,iou,accuracy,recall,epoch,time,avg_power,avg_mem,GFLOPs,energy_kwh,estimated cost (R$)
9,resnet34,6,0.8857,0.9002,0.8204,0.9051,0.9157,13,101m 12s,170.636855,4417.665675,47.3562,0.288897,0.491125
6,resnet18,6,0.8795,0.8986,0.8182,0.9050,0.9191,4,72m 38s,163.421552,3891.377745,9.2239,0.199039,0.338366
0,efficientnet-b0,6,0.8697,0.8977,0.8176,0.9047,0.9280,9,98m 24s,143.988869,7673.067429,90.1533,0.237171,0.403190
3,resnet101,6,0.8968,0.8973,0.8158,0.9047,0.8981,9,193m 49s,169.961673,9319.638611,627.4534,0.550218,0.935371
11,resnet34,All+NDVI,0.8889,0.8938,0.8112,0.9035,0.8994,11,114m 57s,163.939408,4481.455084,47.6992,0.315305,0.536019
7,resnet18,651,0.8825,0.8932,0.8101,0.9019,0.9050,6,76m 5s,164.473232,3917.057108,9.3219,0.209806,0.356669
4,resnet101,651,0.8801,0.8929,0.8096,0.9010,0.9064,23,195m 55s,171.304550,9322.079256,627.5514,0.560430,0.952731
1,efficientnet-b0,651,0.8760,0.8913,0.8070,0.8993,0.9078,34,102m 6s,143.023234,7713.111940,90.1626,0.244405,0.415488
10,resnet34,651,0.8689,0.8907,0.8067,0.9000,0.9145,12,103m 55s,160.364841,4438.792754,47.4542,0.278647,0.473701
2,efficientnet-b0,All+NDVI,0.8652,0.8908,0.8061,0.8974,0.9185,33,114m 1s,124.088729,7748.711429,90.1858,0.236843,0.402633


In [5]:
df['encoder'].value_counts()

encoder
resnet34           3
resnet18           3
efficientnet-b0    3
resnet101          3
Name: count, dtype: int64